In [ ]:
import pandas as pd
import yfinance as yf
import random

def generate_random_price(high, low):
    return random.uniform(low, high)

csi500_l1 = list(pd.read_csv("Yfinance_csi500_list.csv")['Code'])  #https://www.csindex.com.cn/#/indices/family/detail?indexCode=000905 composition with exchange code (SS/SZ) for yfinance
data_frames = []
csi500_l1.remove('601136.SS') # data unavailable

for stock in csi500_l1:
    data_frames.append(yf.download(stock, start="2022-04-01", end="2022-07-31"))

index_data = yf.download("ASHS", start="2022-04-08", end="2022-07-31",period='1d')

# transforming 1d ohlc to 30min close only data using randomness
transformed_df = []

for df in data_frames:
    transformed_data = []
    for index, row in df.iterrows():
        date = index
        open_price = row['Open']
        high_price = row['High']
        low_price = row['Low']
        close_price = row['Close']
        adj_close_price = row['Adj Close']

        # 6 random prices between high and low
        random_prices = [generate_random_price(high_price, low_price) for _ in range(6)]
        # allot values
        data = [
            {'Datetime': date.replace(hour=9, minute=30), 'Close': open_price},  # 1d open price
            {'Datetime': date.replace(hour=10), 'Close': random_prices[0]},
            {'Datetime': date.replace(hour=10, minute=30), 'Close': random_prices[1]},
            {'Datetime': date.replace(hour=11), 'Close': random_prices[2]},
            {'Datetime': date.replace(hour=13, minute=30), 'Close': random_prices[3]},
            {'Datetime': date.replace(hour=14),  'Close': random_prices[4]},
            {'Datetime': date.replace(hour=14, minute=30),  'Close': random_prices[5]},
            {'Datetime': date.replace(hour=14, minute=57), 'Close': close_price} # 1d close price
        ]

        transformed_data.extend(data)

    transformed_df.append(pd.DataFrame(transformed_data))


#combining them
final_df = pd.DataFrame()
final_df['Datetime'] = transformed_df[1]['Datetime']
for i in range(0,len(csi500_l1)):
    final_df[csi500_l1[i]] = transformed_df[i]['Close']

final_df.to_csv('CSI500_data.csv')
index_data.to_csv('CSI500_index.csv')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [ ]:
!pip install baostock -i https://pypi.tuna.tsinghua.edu.cn/simple/ --trusted-host pypi.tuna.tsinghua.edu.cn

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 154.0 kB/s eta 0:00:00


In [ ]:
import baostock as bs
import pandas as pd

# 登陆系统
lg = bs.login()

login success!


In [ ]:

# Get the composition of the CSI500 index
rs = bs.query_zz500_stocks(date='2021-01-01')

# Print the query response information
print('Query error_code:'+rs.error_code)
print('Query error_msg:'+rs.error_msg)

# Initialize an empty list to store the index composition data
zz500_stocks = []

# Fetch the index composition data
while (rs.error_code == '0') & rs.next():
    # Get one row of data and merge it with the previous data
    zz500_stocks.append(rs.get_row_data())

# Create a DataFrame using the fetched data
result = pd.DataFrame(zz500_stocks, columns=rs.fields)


Query error_code:0
Query error_msg:success


In [ ]:
# logout of BaoStock system
bs.logout()

logout success!


In [ ]:
idx_csi500  = result

In [ ]:
idx_csi500.to_csv('idx_csi500_comp.csv')

In [ ]:
code = idx_csi500['code'][0]

In [ ]:
import baostock as bs
import pandas as pd

lg = bs.login()

print('login respond error_code:'+lg.error_code)
print('login respond error_msg:'+lg.error_msg)



login success!
login respond error_code:0
login respond error_msg:success


In [ ]:
idx_csi500 = pd.read_csv("idx_csi500_comp.csv")

In [ ]:
all_stocks = []

for code in idx_csi500['code'] :

    rs = bs.query_history_k_data_plus(code,
        "date,time,code,open,high,low,close,volume,amount,adjustflag",
        start_date='2022-04-01', end_date='2022-07-31',
        frequency="30", adjustflag="3")

    stock = []

    while (rs.error_code == '0') & rs.next():
        stock.append(rs.get_row_data())

    result = pd.DataFrame(stock, columns=rs.fields)

    result = result.fillna(0,inplace = True) # missing value set to 0
    all_stocks.append(result)

In [ ]:
df_list = all_stocks

# CLOSE

all_closes = []
for i in range(0,len(idx_csi500['code'])):
    code = idx_csi500['code'][i]
    df = df_list[i]
    new_df = pd.DataFrame({'Datetime': df['time'], code: df['close']})

    new_df.set_index('Datetime', inplace=True)
    all_closes.append(new_df)

all_closes_df = pd.concat(all_closes, axis=1)
all_closes_df.to_csv('CSI_500_close.csv')

#OPEN

all_closes = []
for i in range(0,len(idx_csi500['code'])):
    code = idx_csi500['code'][i]
    df = df_list[i]
    new_df = pd.DataFrame({'Datetime': df['time'], code: df['open']})
    new_df.set_index('Datetime', inplace=True)
    all_closes.append(new_df)

all_closes_df = pd.concat(all_closes, axis=1)
all_closes_df.to_csv('CSI_500_open.csv')


#HIGH

all_closes = []
for i in range(0,len(idx_csi500['code'])):
    code = idx_csi500['code'][i]
    df = df_list[i]
    new_df = pd.DataFrame({'Datetime': df['time'], code: df['high']})
    new_df.set_index('Datetime', inplace=True)
    all_closes.append(new_df)

all_closes_df = pd.concat(all_closes, axis=1)
all_closes_df.to_csv('CSI_500_high.csv')

#LOW

all_closes = []
for i in range(0,len(idx_csi500['code'])):
    code = idx_csi500['code'][i]
    df = df_list[i]
    new_df = pd.DataFrame({'Datetime': df['time'], code: df['low']})
    new_df.set_index('Datetime', inplace=True)
    all_closes.append(new_df)

all_closes_df = pd.concat(all_closes, axis=1)
all_closes_df.to_csv('CSI_500_low.csv')


#VOLUME

all_closes = []
for i in range(0,len(idx_csi500['code'])):
    code = idx_csi500['code'][i]
    df = df_list[i]
    new_df = pd.DataFrame({'Datetime': df['time'], code: df['volume']})
    new_df.set_index('Datetime', inplace=True)
    all_closes.append(new_df)

all_closes_df = pd.concat(all_closes, axis=1)
all_closes_df.to_csv('CSI_500_volume.csv')

#AMOUNT

all_closes = []
for i in range(0,len(idx_csi500['code'])):
    code = idx_csi500['code'][i]
    df = df_list[i]
    new_df = pd.DataFrame({'Datetime': df['time'], code: df['amount']})
    new_df.set_index('Datetime', inplace=True)
    all_closes.append(new_df)

all_closes_df = pd.concat(all_closes, axis=1)
all_closes_df.to_csv('CSI_500_amount.csv')

In [ ]:
all_stocks[0]

In [ ]:
df_list = all_stocks

# CLOSE

all_closes = []
for i in range(0,len(idx_csi500['code'])):
    code = idx_csi500['code'][i]
    df = df_list[i]
    new_df = pd.DataFrame({'Datetime': df['time'], code: df['close']})

    new_df.set_index('Datetime', inplace=True)
    all_closes.append(new_df)

all_closes_df = pd.concat(all_closes, axis=1)
#all_closes_df.to_csv('CSI_500_close.csv')

all_closes_df

TypeError: ignored